## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1412. Find the Quiet Students in All Exam (Hard)**

**Table: Student**

| Column Name         | Type    |
|---------------------|---------|
| student_id          | int     |
| student_name        | varchar |

student_id is the primary key (column with unique values) for this table.
student_name is the name of the student.
 
**Table: Exam**

| Column Name   | Type    |
|---------------|---------|
| exam_id       | int     |
| student_id    | int     |
| score         | int     |

(exam_id, student_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates that the student with student_id had a score points in the exam with id exam_id.

A quiet student is the one who took at least one exam and did not score the highest or the lowest score.

**Write a solution to report the students (student_id, student_name) being quiet in all exams. Do not return the student who has never taken any exam.**

Return the result table ordered by student_id.

The result format is in the following example.

**Example 1:**

**Input:** 

**Student table:**
| student_id  | student_name  |
|-------------|---------------|
| 1           | Daniel        |
| 2           | Jade          |
| 3           | Stella        |
| 4           | Jonathan      |
| 5           | Will          |

**Exam table:**

| exam_id    | student_id   | score     |
|------------|--------------|-----------|
| 10         |     1        |    70     |
| 10         |     2        |    80     |
| 10         |     3        |    90     |
| 20         |     1        |    80     |
| 30         |     1        |    70     |
| 30         |     3        |    80     |
| 30         |     4        |    90     |
| 40         |     1        |    60     |
| 40         |     2        |    70     |
| 40         |     4        |    80     |

**Output:** 
| student_id  | student_name  |
|-------------|---------------|
| 2           | Jade          |

**Explanation:** 
- For exam 1: Student 1 and 3 hold the lowest and high scores respectively.
- For exam 2: Student 1 hold both highest and lowest score.
- For exam 3 and 4: Studnet 1 and 4 hold the lowest and high scores respectively.
- Student 2 and 5 have never got the highest or lowest in any of the exams.
- Since student 5 is not taking any exam, he is excluded from the result. So, we only return the information of Student 2.

In [0]:
students_data_1412 = [
    (1, "Daniel"),
    (2, "Jade"),
    (3, "Stella"),
    (4, "Jonathan"),
    (5, "Will")
]
students_columns_1412 = ["student_id", "student_name"]
students_df_1412 = spark.createDataFrame(students_data_1412, students_columns_1412)
students_df_1412.show()

exams_data_1412 = [
    (10, 1, 70),
    (10, 2, 80),
    (10, 3, 90),
    (20, 1, 80),
    (30, 1, 70),
    (30, 3, 80),
    (30, 4, 90),
    (40, 1, 60),
    (40, 2, 70),
    (40, 4, 80)
]
exams_columns_1412 = ["exam_id", "student_id", "score"]
exams_df_1412 = spark.createDataFrame(exams_data_1412, exams_columns_1412)
exams_df_1412.show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|      Daniel|
|         2|        Jade|
|         3|      Stella|
|         4|    Jonathan|
|         5|        Will|
+----------+------------+

+-------+----------+-----+
|exam_id|student_id|score|
+-------+----------+-----+
|     10|         1|   70|
|     10|         2|   80|
|     10|         3|   90|
|     20|         1|   80|
|     30|         1|   70|
|     30|         3|   80|
|     30|         4|   90|
|     40|         1|   60|
|     40|         2|   70|
|     40|         4|   80|
+-------+----------+-----+



In [0]:
exam_scores_extremes_1412 = exams_df_1412\
                            .groupBy("exam_id").agg(
                                    max("score").alias("max_score"),
                                    min("score").alias("min_score")
                            )

In [0]:
extreme_students_1412 = exams_df_1412\
                        .join(exam_scores_extremes_1412, on="exam_id") \
                            .filter((col("score") == col("max_score")) | (col("score") == col("min_score"))) \
                                .select("student_id").distinct()

In [0]:
students_with_exams_1412 = exams_df_1412.select("student_id").distinct()

In [0]:
quiet_students_1412 = students_with_exams_1412.join(extreme_students_1412, on="student_id", how="left_anti")

In [0]:
quiet_students_1412\
    .join(students_df_1412, on="student_id") \
    .select("student_id", "student_name") \
    .orderBy("student_id").show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         2|        Jade|
+----------+------------+

